In [1]:
# IMPLEMENTING ML MODELS FOR AIRBNB DATA TO PREDICT PRICES

In [2]:
#import dependecies

import pandas as pd
import numpy as np

In [3]:
#data cleaning for nyc
pd.set_option('display.max_columns',None)

review_df= pd.read_csv('review_polarity_NYC.csv')

nyc_df = pd.read_csv('listings-nyc.csv')

nyc_df["room_type"].unique()
p1 = pd.get_dummies(nyc_df["room_type"])
nyc_df["neighbourhood_group_cleansed"].unique()
p2 = pd.get_dummies(nyc_df["neighbourhood_group_cleansed"])
p = pd.concat([p1, p2], axis = 1)
nyc_df = pd.concat([nyc_df, p], axis = 1)
nyc_df["host_response_rate"] = nyc_df["host_response_rate"].apply(lambda x: str(x)[:-1] if str(x) != "nan" else x).astype("float64")
nyc_df["host_acceptance_rate"] = nyc_df["host_acceptance_rate"].apply(lambda x: str(x)[:-1] if str(x) != "nan" else x).astype("float64")
nyc_df["host_is_superhost"] = nyc_df["host_is_superhost"].apply(lambda x: 1 if str(x) == "t" else 0 if str(x) == "f" else x).astype("float64")
nyc_df["total_verifications"] = nyc_df["host_verifications"].apply(lambda x: len(x.replace("[", "").replace("]", "").split(",")))
nyc_df["host_has_profile_pic"] = nyc_df["host_has_profile_pic"].apply(lambda x: 1 if str(x) == "t" else 0 if str(x) == "f" else x).astype("float64")
nyc_df["host_identity_verified"] = nyc_df["host_identity_verified"].apply(lambda x: 1 if str(x) == "t" else 0 if str(x) == "f" else x).astype("float64")
nyc_df["instant_bookable"] = nyc_df["instant_bookable"].apply(lambda x: 1 if str(x) == "t" else 0).astype("float64")
nyc_df["bathrooms_text"].unique()
nyc_df["bathrooms_text"]=nyc_df["bathrooms_text"].str.replace('half-bath','0.5')
nyc_df["bathrooms"]=nyc_df['bathrooms_text'].str.extract('([0-9][,.]*[0-9]*)') 
nyc_df["bathrooms_text"]=nyc_df["bathrooms_text"].str.contains('shared', regex=True)
nyc_df["shared_bathrooms"]=nyc_df["bathrooms_text"].apply(lambda x: 1 if x == True else 0).astype("float64")
nyc_df["total_amenities"] = nyc_df["amenities"].apply(lambda x: len(x.replace("[", "").replace("]", "").split(",")))
nyc_df["price"]=nyc_df["price"].replace({'\$':''}, regex = True)
nyc_df["price"]=pd.to_numeric(nyc_df["price"], errors='coerce')

cols_nyc=['host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 
        'total_verifications', 'total_amenities', 'Entire home/apt',
       'Hotel room', 'Private room', 'Shared room', 'Bronx', 'Brooklyn',
       'Manhattan', 'Queens', 'Staten Island', 'instant_bookable', 
        "host_listings_count", "host_total_listings_count", "accommodates", "bedrooms", "beds","bathrooms","shared_bathrooms", 
 "availability_30", "availability_60", "availability_90", "availability_365",
 "minimum_nights", "number_of_reviews", 'number_of_reviews_ltm',
       'number_of_reviews_l30d',"reviews_per_month", "review_scores_rating"]

polarity_df = review_df.groupby(by = ["listing_id"])["polarity"].mean().reset_index()
polarity_df["id"] = polarity_df["listing_id"].astype("float64")
polarity_df.drop(columns = ["listing_id"], inplace= True)

merged_df = pd.merge(nyc_df, polarity_df, on = "id", how = "inner")
merged_df[cols_nyc]=merged_df[cols_nyc].fillna(0)
merged_df["price"]=merged_df["price"].fillna(0)
df=merged_df[(merged_df["price"]< 500) & (merged_df["price"]!=0)]
df[cols_nyc]


/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,total_verifications,total_amenities,Entire home/apt,Hotel room,Private room,Shared room,Bronx,Brooklyn,Manhattan,Queens,Staten Island,instant_bookable,host_listings_count,host_total_listings_count,accommodates,bedrooms,beds,bathrooms,shared_bathrooms,availability_30,availability_60,availability_90,availability_365,minimum_nights,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,reviews_per_month,review_scores_rating
0,35.0,18.0,0.0,1.0,1.0,9,31,1,0,0,0,0,0,1,0,0,0.0,8.0,8.0,1,0.0,1.0,1,0.0,0,17,47,322,30,48,0,0,0.33,4.70
1,21.0,76.0,0.0,1.0,1.0,6,35,1,0,0,0,0,1,0,0,0,0.0,1.0,1.0,3,1.0,3.0,1,0.0,4,10,33,220,1,408,38,0,4.91,4.46
2,100.0,100.0,0.0,1.0,1.0,8,5,0,0,1,0,0,1,0,0,0,0.0,1.0,1.0,2,1.0,1.0,0,0.0,30,60,90,365,30,50,0,0,0.53,4.52
3,100.0,25.0,0.0,1.0,1.0,3,14,1,0,0,0,0,1,0,0,0,0.0,1.0,1.0,4,2.0,2.0,1.5,0.0,0,2,3,91,5,2,1,0,0.02,5.00
4,100.0,100.0,0.0,1.0,0.0,4,25,0,0,1,0,0,0,1,0,0,0.0,1.0,1.0,2,1.0,1.0,1,0.0,4,4,26,218,2,505,31,4,3.70,4.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25977,100.0,91.0,0.0,1.0,1.0,1,37,0,0,1,0,0,0,0,1,0,0.0,2.0,2.0,2,1.0,2.0,1,1.0,28,52,82,356,2,8,8,1,6.86,4.75
25978,89.0,100.0,0.0,1.0,1.0,3,21,0,0,1,0,0,0,1,0,0,1.0,1.0,1.0,2,1.0,1.0,1,0.0,12,39,39,39,1,9,9,3,4.03,4.56
25979,89.0,100.0,0.0,1.0,1.0,3,21,0,0,1,0,0,0,1,0,0,1.0,1.0,1.0,2,1.0,1.0,1,0.0,6,26,26,26,1,17,17,6,10.63,4.53
25980,89.0,100.0,0.0,1.0,1.0,3,21,0,0,1,0,0,0,1,0,0,1.0,1.0,1.0,2,1.0,1.0,1,0.0,18,46,46,46,1,7,7,2,7.00,4.86


In [4]:
#data cleaning for la

reviewLA_df= pd.read_csv('reviews_polarity_LA.csv')
la_df=pd.read_csv('listings-la.csv')

la_df["room_type"].unique()
p1 = pd.get_dummies(la_df["room_type"])
la_df["neighbourhood_group_cleansed"].unique()
p2 = pd.get_dummies(la_df["neighbourhood_group_cleansed"])
p = pd.concat([p1, p2], axis = 1)
la_df = pd.concat([la_df, p], axis = 1)
la_df["host_response_rate"] = la_df["host_response_rate"].apply(lambda x: str(x)[:-1] if str(x) != "nan" else x).astype("float64")
la_df["host_acceptance_rate"] = la_df["host_acceptance_rate"].apply(lambda x: str(x)[:-1] if str(x) != "nan" else x).astype("float64")
la_df["host_is_superhost"] = la_df["host_is_superhost"].apply(lambda x: 1 if str(x) == "t" else 0 if str(x) == "f" else x).astype("float64")
la_df["total_verifications"] = la_df["host_verifications"].apply(lambda x: len(x.replace("[", "").replace("]", "").split(",")))
la_df["host_has_profile_pic"] = la_df["host_has_profile_pic"].apply(lambda x: 1 if str(x) == "t" else 0 if str(x) == "f" else x).astype("float64")
la_df["host_identity_verified"] = la_df["host_identity_verified"].apply(lambda x: 1 if str(x) == "t" else 0 if str(x) == "f" else x).astype("float64")
la_df["instant_bookable"] = la_df["instant_bookable"].apply(lambda x: 1 if str(x) == "t" else 0).astype("float64")
la_df["bathrooms_text"].unique()
la_df["bathrooms_text"]=la_df["bathrooms_text"].str.replace('half-bath','0.5')
la_df["bathrooms"]=la_df['bathrooms_text'].str.extract('([0-9][,.]*[0-9]*)') 
la_df["bathrooms_text"]=la_df["bathrooms_text"].str.contains('shared', regex=True)
la_df["shared_bathrooms"]=la_df["bathrooms_text"].apply(lambda x: 1 if x == True else 0).astype("float64")
la_df["total_amenities"] = la_df["amenities"].apply(lambda x: len(x.replace("[", "").replace("]", "").split(",")))
la_df["price"]=la_df["price"].replace({'\$':''}, regex = True)
la_df["price"]=pd.to_numeric(la_df["price"], errors='coerce')

cols_la = ['host_is_superhost', 'instant_bookable', 'host_total_listings_count', 
        'total_verifications', 
        'City of Los Angeles',
        'Shared room', 'Private room', 'Hotel room', 'Entire home/apt',
         #'Other Cities', 'Unincorporated Areas',
        'total_amenities', 
        'accommodates','bedrooms', 'beds', 'bathrooms','shared_bathrooms',
        "availability_30", "availability_365", 
        "minimum_nights", "number_of_reviews"]

polarityla_df = reviewLA_df.groupby(by = ["listing_id"])["polarity"].mean().reset_index()
polarityla_df["id"] = polarityla_df["listing_id"].astype("float64")
polarityla_df.drop(columns = ["listing_id"], inplace= True)

mergedla_df = pd.merge(la_df, polarityla_df, on = "id", how = "inner")
mergedla_df[cols_la]=mergedla_df[cols_la].fillna(0)
mergedla_df["price"]=mergedla_df["price"].fillna(0)
df2=mergedla_df[(mergedla_df["price"]< 500) & (mergedla_df["price"]!=0)]
df2[cols_la]


/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (58,59) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,host_is_superhost,instant_bookable,host_total_listings_count,total_verifications,City of Los Angeles,Shared room,Private room,Hotel room,Entire home/apt,total_amenities,accommodates,bedrooms,beds,bathrooms,shared_bathrooms,availability_30,availability_365,minimum_nights,number_of_reviews
0,0.0,0.0,1.0,8,0,0,0,0,1,25,6,2.0,3.0,2,0.0,0,193,30,2
1,1.0,1.0,2.0,7,1,0,1,0,0,40,1,1.0,1.0,1,1.0,6,300,30,33
2,1.0,0.0,2.0,8,0,0,1,0,0,19,1,1.0,1.0,1,0.0,22,357,7,21
3,0.0,0.0,6.0,5,1,0,0,0,1,21,2,1.0,2.0,1,0.0,0,274,31,23
4,1.0,0.0,8.0,9,1,0,1,0,0,24,1,1.0,1.0,1,1.0,0,147,30,312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22656,0.0,1.0,0.0,2,0,0,0,0,1,60,3,1.0,2.0,1,0.0,26,333,2,10
22657,1.0,1.0,213.0,5,1,0,1,0,0,65,3,1.0,2.0,2.5,0.0,30,365,30,1
22658,0.0,1.0,0.0,2,0,0,0,0,1,18,4,2.0,3.0,1,0.0,6,319,1,1
22659,1.0,1.0,7.0,9,0,0,0,0,1,42,4,2.0,3.0,1,0.0,1,306,1,5


In [5]:
# NYC 
X_nyc=pd.DataFrame(df[cols_nyc]).to_numpy()
y_nyc=pd.DataFrame(df["price"]).to_numpy()
y_nyc=np.log(y_nyc)
print(X_nyc.shape)
print(y_nyc.shape)

nyc_train= 25000

# LA
X_la=pd.DataFrame(df2[cols_la]).to_numpy()
y_la=pd.DataFrame(df2["price"]).to_numpy()
y_la=np.log(y_la)
print(X_la.shape)
print(y_la.shape)

la_train = 20000

(25218, 34)
(25218, 1)
(21149, 19)
(21149, 1)


In [6]:
#LINEAR REGRESSION FOR NYC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

n_samples, n_features = nyc_train,X_nyc.shape[1]
reg = LinearRegression()  
reg.fit(X_nyc[:nyc_train], y_nyc[:nyc_train])  
y_nyc_pred = reg.predict(X_nyc[nyc_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_nyc[nyc_train:], y_nyc_pred))
print('Coefficients: \n', reg.coef_)
print("R2:", r2_score(y_nyc[nyc_train:,:], y_nyc_pred))

Mean squared error: 0.12
Coefficients: 
 [[ 1.69156290e-04 -6.40932115e-04  4.53302810e-02 -4.19613868e-02
   4.57432749e-02 -6.40930684e-03  5.55518102e-03 -4.46274818e+09
  -4.46274818e+09 -4.46274818e+09 -4.46274818e+09 -1.72800105e+09
  -1.72800105e+09 -1.72800105e+09 -1.72800105e+09 -1.72800105e+09
   3.20761242e-02  4.91985580e+10 -4.91985580e+10  7.51019284e-02
   9.01228528e-02 -1.25965194e-02  5.29606264e-02 -2.50823028e-01
   1.19576366e-02 -1.04882814e-03  8.07650471e-04 -1.29281482e-04
  -1.21453498e-03 -4.04719538e-04  2.07524615e-04  3.16239593e-03
  -1.77656463e-03  2.30882391e-02]]
R2: 0.6253145843533956


In [7]:
#LINEAR REGRESSION FOR LA

n_samples, n_features = la_train,X_la.shape[1]
reg = LinearRegression()             
reg.fit(X_la[:la_train], y_la[:la_train])  
y_la_pred = reg.predict(X_la[la_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_la[la_train:], y_la_pred))
print('Coefficients: \n', reg.coef_)
print("R2:", r2_score(y_la[la_train:,:], y_la_pred))

Mean squared error: 0.14
Coefficients: 
 [[ 7.21709927e-02 -2.62854791e-02  1.95051915e-04 -2.23586004e-03
  -2.85695950e-04 -6.42263602e-01 -4.99319788e-02  3.18828896e-01
   3.73366684e-01  4.39146120e-03  5.74160783e-02  1.98811917e-01
  -2.85214238e-02  3.51377885e-02 -3.08594716e-01  1.07839709e-02
  -7.62082012e-05 -8.90649286e-04  2.62271378e-05]]
R2: 0.6513050699772612


In [8]:
# RIDGE REGRESSION FOR NYC

from sklearn.linear_model import Ridge

n_samples, n_features = nyc_train,X_nyc.shape[1]
rdg = Ridge(alpha = 7)  #
rdg.fit(X_nyc[:nyc_train], y_nyc[:nyc_train])  
y_nyc_pred = rdg.predict(X_nyc[nyc_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_nyc[nyc_train:], y_nyc_pred))
print('Coefficients: \n', rdg.coef_)
print("R2:", r2_score(y_nyc[nyc_train:,:], y_nyc_pred))


Mean squared error: 0.12
Coefficients: 
 [[ 1.69314608e-04 -6.19779214e-04  4.20068206e-02 -2.05051866e-02
   4.33870692e-02 -5.21625892e-03  5.35017298e-03  1.94731604e-01
   4.49577968e-01 -2.13961097e-01 -4.30348475e-01 -1.57811029e-01
   7.87380746e-02  3.47104991e-01 -8.38320444e-02 -1.84199992e-01
   3.47844740e-02  1.60935579e-04  1.60935583e-04  7.72221178e-02
   9.29886499e-02 -9.54639198e-03  5.22241243e-02 -2.52712196e-01
   1.16957130e-02 -8.42495605e-04  7.71345692e-04 -1.35463082e-04
  -1.21640340e-03 -4.09585898e-04  3.00408092e-04  1.98655625e-03
  -1.78339269e-03  2.59666366e-02]]
R2: 0.6259762098615915


In [9]:
#RIDGE REGRESSION FOR LA

n_samples, n_features = la_train,X_la.shape[1]
rdg = Ridge(alpha = 10)             
rdg.fit(X_la[:la_train], y_la[:la_train])  
y_la_pred = rdg.predict(X_la[la_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_la[la_train:], y_la_pred))
print('Coefficients: \n', rdg.coef_)
print("R2:", r2_score(y_la[la_train:,:], y_la_pred))


Mean squared error: 0.14
Coefficients: 
 [[ 7.20043768e-02 -2.58302774e-02  1.95125133e-04 -2.23676435e-03
  -5.45093174e-04 -6.19256460e-01 -4.57068838e-02  2.88660148e-01
   3.76303196e-01  4.40806491e-03  5.76590232e-02  1.98825900e-01
  -2.88125529e-02  3.46856564e-02 -3.10131420e-01  1.07739878e-02
  -7.63702049e-05 -8.90847311e-04  2.74333640e-05]]
R2: 0.6515025926821696


In [11]:
# SVM for NYC
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

n_samples, n_features = nyc_train,X_nyc.shape[1]
regr = make_pipeline(StandardScaler(),LinearSVR(random_state=0, tol=1e-5))
regr.fit(X_nyc[:nyc_train], y_nyc[:nyc_train])
y_nyc_pred = regr.predict(X_nyc[nyc_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_nyc[nyc_train:], y_nyc_pred))
print("R2:", r2_score(y_nyc[nyc_train:,:], y_nyc_pred))

/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean squared error: 0.12
R2: 0.6290625041298277


/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [12]:
#SVM for LA
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

n_samples, n_features = la_train,X_la.shape[1]
regr = make_pipeline(StandardScaler(),LinearSVR(random_state=0, tol=1e-5))
regr.fit(X_la[:la_train], y_la[:la_train])
y_la_pred = regr.predict(X_la[la_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_la[la_train:], y_la_pred))
print("R2:", r2_score(y_la[la_train:,:], y_la_pred))


/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean squared error: 0.15
R2: 0.6281843609777324


/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [13]:
#RANDOM FOREST REGRESSOR FOR NYC
from sklearn.ensemble import RandomForestRegressor

n_samples, n_features = nyc_train,X_nyc.shape[1]
regr = RandomForestRegressor(max_depth=9, random_state=42)
regr.fit(X_nyc[:nyc_train], y_nyc[:nyc_train])
y_nyc_pred = regr.predict(X_nyc[nyc_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_nyc[nyc_train:], y_nyc_pred))
print("R2:", r2_score(y_nyc[nyc_train:,:], y_nyc_pred))

/var/folders/mf/r9n0tb5s5_d0kp6p05vwrfqm0000gn/T/ipykernel_23960/3178122885.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(X_nyc[:nyc_train], y_nyc[:nyc_train])


Mean squared error: 0.10
R2: 0.6866250744483109


In [14]:
#RANDOM FOREST REGRESSOR FOR LA

n_samples, n_features = la_train,X_nyc.shape[1]
regr = RandomForestRegressor(max_depth=11, random_state=42)
regr.fit(X_la[:la_train], y_la[:la_train])
y_la_pred = regr.predict(X_la[la_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_la[la_train:], y_la_pred))
print("R2:", r2_score(y_la[la_train:,:], y_la_pred))

/var/folders/mf/r9n0tb5s5_d0kp6p05vwrfqm0000gn/T/ipykernel_23960/2039485699.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(X_la[:la_train], y_la[:la_train])


Mean squared error: 0.12
R2: 0.702117446543984


In [15]:
#XG BOOST FOR NYC
from sklearn import ensemble
params = {
    "n_estimators": 300,
    "max_depth": 3,
    "min_samples_split": 5,
    "learning_rate": 0.08,
    "loss": "squared_error",
}
regr = ensemble.GradientBoostingRegressor(**params)
regr.fit(X_nyc[:nyc_train], y_nyc[:nyc_train])
y_nyc_pred = regr.predict(X_nyc[nyc_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_nyc[nyc_train:], y_nyc_pred))
print("R2:", r2_score(y_nyc[nyc_train:,:], y_nyc_pred))

/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean squared error: 0.10
R2: 0.685949966846576


In [16]:
#XG BOOST FOR LA
params = {
    "n_estimators": 300,
    "max_depth": 3,
    "min_samples_split": 5,
    "learning_rate": 0.08,
    "loss": "squared_error",
}
regr = ensemble.GradientBoostingRegressor(**params)
regr.fit(X_la[:la_train], y_la[:la_train])
y_la_pred = regr.predict(X_la[la_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_la[la_train:], y_la_pred))
print("R2:", r2_score(y_la[la_train:,:], y_la_pred))

/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean squared error: 0.11
R2: 0.7243116745619331
